# Modelo de Predicción de Gasto Semanal con LSTM

Este notebook implementa un modelo de predicción de gasto semanal utilizando redes neuronales recurrentes LSTM (Long Short-Term Memory).
La predicción se realiza con periodos de 1 semana, como se especificó en los requerimientos.


In [7]:
# --- 1. IMPORTAR LIBRERÍAS --------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import holidays

# Librerías para deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from fastapi import FastAPI

# Configurar semilla para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)


In [8]:
# --- 2. CARGAR Y PREPROCESAR DATOS --------------------------------------
# Cargar datos de clientes
clients = pd.read_csv('../data/raw/base_clientes_final.csv')
clients.info()

# Cargar datos de transacciones
txn = pd.read_csv('../data/raw/base_transacciones_final.csv')
txn.info()

# Convertir fecha a datetime y crear columna de semana
txn["fecha"] = pd.to_datetime(txn["fecha"])
txn["week"] = txn["fecha"].dt.to_period("W")

# Agregar datos por cliente y semana
agg = (txn.groupby(["id", "week"])
           .agg(spend=("monto", "sum"),
                n_tx=("monto", "size"),
                max_tx=("monto", "max"),
                avg_ticket=("monto", "mean"))
           .reset_index())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     1000 non-null   object
 1   fecha_nacimiento       1000 non-null   object
 2   fecha_alta             1000 non-null   object
 3   id_municipio           1000 non-null   int64 
 4   id_estado              1000 non-null   int64 
 5   tipo_persona           1000 non-null   object
 6   genero                 1000 non-null   object
 7   actividad_empresarial  1000 non-null   object
dtypes: int64(2), object(6)
memory usage: 62.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346011 entries, 0 to 346010
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             346011 non-null  object 
 1   fecha          346011 non-null  object 
 2   comercio       346011 non-null  object 

In [9]:
# --- 3. COMPLETAR CALENDARIO --------------------------------------------
# Crear un índice completo para todas las semanas y clientes
full_idx = pd.MultiIndex.from_product(
    [agg["id"].unique(),
     pd.period_range(agg["week"].min(), agg["week"].max(), freq="W")],
    names=["id", "week"]
)
panel = (agg.set_index(["id", "week"])
             .reindex(full_idx, fill_value=0)
             .reset_index())


In [10]:
# --- 4. DATOS ESTÁTICOS DE CLIENTES -------------------------------------------
# Fecha fija para reproducibilidad
today = pd.Timestamp("2025-05-24")
# Calcular edad y antigüedad en meses
clients["age"] = ((today - pd.to_datetime(clients["fecha_nacimiento"])).dt.days // 365)
clients["tenure_months"] = ((today - pd.to_datetime(clients["fecha_alta"])).dt.days // 30)

# Seleccionar columnas estáticas y unir con panel
static_cols = ["id", "age", "tenure_months", "id_estado",
               "tipo_persona", "genero", "actividad_empresarial"]
panel = panel.merge(clients[static_cols], on="id", how="left")


In [11]:
# --- 5. CARACTERÍSTICAS DE REZAGO (LAG FEATURES) -------------------------------------------------
panel = panel.sort_values(["id", "week"])
# Crear variables de rezago para gasto y otras métricas
for k in range(1, 7):
    panel[f"spend_lag{k}"] = panel.groupby("id")["spend"].shift(k)

# También crear rezagos para número de transacciones y ticket promedio
for k in range(1, 4):
    panel[f"n_tx_lag{k}"] = panel.groupby("id")["n_tx"].shift(k)
    panel[f"avg_ticket_lag{k}"] = panel.groupby("id")["avg_ticket"].shift(k)

# Crear variables de media móvil y estadísticas
panel["rolling_mean_3"] = panel.groupby("id")["spend"].rolling(3).mean().reset_index(level=0, drop=True)
panel["rolling_mean_6"] = panel.groupby("id")["spend"].rolling(6).mean().reset_index(level=0, drop=True)
panel["rolling_std_3"] = panel.groupby("id")["spend"].rolling(3).std().reset_index(level=0, drop=True)
panel["rolling_std_6"] = panel.groupby("id")["spend"].rolling(6).std().reset_index(level=0, drop=True)
panel["rolling_max_3"] = panel.groupby("id")["spend"].rolling(3).max().reset_index(level=0, drop=True)
panel["rolling_min_3"] = panel.groupby("id")["spend"].rolling(3).min().reset_index(level=0, drop=True)

# Calcular tendencias y tasas de cambio
panel["spend_trend"] = panel["spend"] - panel["rolling_mean_6"]
panel["spend_pct_change"] = panel.groupby("id")["spend"].pct_change()
# Reemplazar infinitos con NaN en pct_change
panel["spend_pct_change"].replace([np.inf, -np.inf], np.nan, inplace=True)
panel["spend_acceleration"] = panel.groupby("id")["spend_pct_change"].diff()
# Reemplazar infinitos con NaN en acceleration
panel["spend_acceleration"].replace([np.inf, -np.inf], np.nan, inplace=True)

# Volatilidad relativa (coeficiente de variación)
panel["spend_volatility"] = panel["rolling_std_6"] / (panel["rolling_mean_6"] + 1)  # +1 para evitar división por cero


In [12]:
# --- 6. ESTACIONALIDAD Y DÍAS FESTIVOS --------------------------------------
# Características cíclicas para la semana del año
panel["week_of_year"] = panel["week"].dt.week
panel["week_sin"] = np.sin(2 * np.pi * panel["week_of_year"] / 52)
panel["week_cos"] = np.cos(2 * np.pi * panel["week_of_year"] / 52)

# También mantener información del mes para estacionalidad mensual
panel["month"] = panel["week"].dt.asfreq('M')
panel["month_idx"] = panel["month"].dt.month
panel["month_sin"] = np.sin(2 * np.pi * panel["month_idx"] / 12)
panel["month_cos"] = np.cos(2 * np.pi * panel["month_idx"] / 12)

# Agregar fechas importantes en México
def is_buen_fin(date): return date.month == 11 and date.day >= 15
def is_navidad(date): return date.month == 12 and date.day >= 20
def is_mothers_day(date): return date.month == 5 and date.day == 10

txn["buen_fin"] = txn["fecha"].apply(is_buen_fin)
txn["navidad"] = txn["fecha"].apply(is_navidad)
txn["dia_madre"] = txn["fecha"].apply(is_mothers_day)

# Agregar variables de días festivos al panel
holiday_agg = txn.groupby(["id", txn["fecha"].dt.to_period("W")])[["buen_fin", "navidad", "dia_madre"]].sum().reset_index()
holiday_agg.rename(columns={"fecha": "week"}, inplace=True)
panel = panel.merge(holiday_agg, on=["id", "week"], how="left").fillna(0)

# Agregar días festivos de México usando el paquete holidays
def add_holiday_feature(df):
    """Agrega una columna is_holiday al DataFrame"""
    # Convertir week a datetime para poder comparar con holidays
    df['week_start'] = df['week'].dt.start_time

    # Obtener días festivos de México para los años relevantes
    start_year = df['week_start'].dt.year.min()
    end_year = df['week_start'].dt.year.max()
    mx_holidays = holidays.Mexico(years=range(start_year, end_year + 1))

    # Marcar semanas que contienen días festivos
    df['is_holiday'] = df['week_start'].apply(
        lambda x: any(x <= pd.Timestamp(day) <= x + pd.Timedelta(days=6) for day in mx_holidays if start_year <= day.year <= end_year)
    ).astype(int)

    # Eliminar columna temporal
    df.drop('week_start', axis=1, inplace=True)
    return df

panel = add_holiday_feature(panel)

# Agregar indicador de fin de semana (aunque para datos semanales no es tan relevante)
panel['is_weekend'] = 1  # Todas las semanas tienen fin de semana


In [13]:
# --- 7. VARIABLE OBJETIVO Y PREPROCESAMIENTO FINAL -------------------------------------------------------
# La variable objetivo es el gasto de la próxima semana
panel["spend_next"] = panel.groupby("id")["spend"].shift(-1)

# Crear características de interacción
panel["spend_by_age"] = panel["spend"] / (panel["age"] + 1)  # +1 para evitar división por cero
panel["spend_by_tenure"] = panel["spend"] / (panel["tenure_months"] + 1)
panel["tx_frequency"] = panel["n_tx"] / (panel["tenure_months"] + 1)
panel["spend_per_tx"] = panel["spend"] / (panel["n_tx"] + 1)

# Características no lineales
panel["spend_squared"] = panel["spend"] ** 2
panel["log_spend"] = np.log1p(panel["spend"])  # log(1+x) para manejar ceros

# Relación entre gasto actual y promedio histórico
panel["spend_vs_history"] = panel["spend"] / (panel["rolling_mean_6"] + 1)

# Manejo de valores faltantes para características numéricas
numeric_cols = panel.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    if col != "spend_next" and panel[col].isnull().sum() > 0:
        # Imputar con la mediana por cliente, o la mediana global si no hay datos del cliente
        panel[col] = panel.groupby("id")[col].transform(
            lambda x: x.fillna(x.median() if not pd.isna(x.median()) else panel[col].median())
        )

# Eliminar filas sin valores para la variable objetivo después de la imputación
panel = panel.dropna(subset=["spend_next"])


In [14]:
# --- 8. PREPARACIÓN DE DATOS PARA LSTM --------------------------------------------------------
# Definir columnas de características
feature_cols = [
    # Características básicas de transacciones
    "spend", "n_tx", "max_tx", "avg_ticket",

    # Características de rezago para gasto
    "spend_lag1", "spend_lag2", "spend_lag3", "spend_lag4", "spend_lag5", "spend_lag6",

    # Características de rezago para otras métricas
    "n_tx_lag1", "n_tx_lag2", "n_tx_lag3",
    "avg_ticket_lag1", "avg_ticket_lag2", "avg_ticket_lag3",

    # Estadísticas móviles
    "rolling_mean_3", "rolling_mean_6", 
    "rolling_std_3", "rolling_std_6",
    "rolling_max_3", "rolling_min_3",

    # Tendencias y cambios
    "spend_trend", "spend_pct_change", "spend_acceleration", "spend_volatility",

    # Características de interacción
    "spend_by_age", "spend_by_tenure", "tx_frequency", "spend_per_tx",
    "spend_vs_history",

    # Transformaciones no lineales
    "spend_squared", "log_spend",

    # Características demográficas
    "age", "tenure_months",

    # Características estacionales
    "week_sin", "week_cos", "month_sin", "month_cos",
    "buen_fin", "navidad", "dia_madre", "is_holiday", "is_weekend"
]

# Convertir variables categóricas a one-hot encoding
cat_features = ["id_estado", "tipo_persona", "genero", "actividad_empresarial"]
panel_encoded = pd.get_dummies(panel, columns=cat_features, drop_first=True)

# Actualizar lista de características con columnas one-hot
for col in panel_encoded.columns:
    if any(col.startswith(f"{cat}_") for cat in cat_features):
        feature_cols.append(col)

# División cronológica (mismo corte para todos los clientes)
# Para datos semanales, usamos las últimas 12 semanas como validación y test
train_mask = panel_encoded["week"] <= panel_encoded["week"].max() - 12   # todas las semanas excepto las últimas 12
val_mask = (panel_encoded["week"] > panel_encoded["week"].max() - 12) & (panel_encoded["week"] <= panel_encoded["week"].max() - 4)  # 8 semanas para validación
test_mask = panel_encoded["week"] > panel_encoded["week"].max() - 4      # últimas 4 semanas para test

# Función para crear secuencias para LSTM
def create_sequences(data, client_ids, seq_length=6):
    """
    Crea secuencias de datos para LSTM agrupadas por cliente.

    Args:
        data: DataFrame con los datos
        client_ids: Lista de IDs de clientes a incluir
        seq_length: Longitud de la secuencia (número de semanas anteriores)

    Returns:
        X: Secuencias de entrada (samples, time steps, features)
        y: Valores objetivo
        client_map: Mapeo de índices a IDs de clientes
    """
    X, y = [], []
    client_map = []

    for client_id in client_ids:
        # Filtrar datos del cliente
        client_data = data[data['id'] == client_id].sort_values('week')

        if len(client_data) <= seq_length:
            continue

        # Extraer características y objetivo
        features = client_data[feature_cols].values
        target = client_data['spend_next'].values

        # Crear secuencias
        for i in range(len(features) - seq_length):
            X.append(features[i:i+seq_length])
            y.append(target[i+seq_length-1])
            client_map.append(client_id)

    return np.array(X), np.array(y), client_map

# Obtener listas de clientes para cada conjunto
train_clients = panel_encoded.loc[train_mask, 'id'].unique()
val_clients = panel_encoded.loc[val_mask, 'id'].unique()
test_clients = panel_encoded.loc[test_mask, 'id'].unique()

# Crear secuencias para entrenamiento, validación y prueba
X_train, y_train, train_client_map = create_sequences(panel_encoded, train_clients)
X_val, y_val, val_client_map = create_sequences(panel_encoded, val_clients)
X_test, y_test, test_client_map = create_sequences(panel_encoded, test_clients)

print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de X_val: {X_val.shape}")
print(f"Forma de X_test: {X_test.shape}")


Forma de X_train: (51000, 6, 106)
Forma de X_val: (51000, 6, 106)
Forma de X_test: (51000, 6, 106)


In [15]:
# --- 9. NORMALIZACIÓN DE DATOS --------------------------------------
# Normalizar características para mejorar el entrenamiento del LSTM
# Reshape para aplicar el scaler
n_samples_train, n_timesteps, n_features = X_train.shape
X_train_reshaped = X_train.reshape(n_samples_train * n_timesteps, n_features)

# Función para limpiar datos antes de escalar
def clean_data_for_scaling(data):
    # Reemplazar infinitos y NaN con valores finitos
    data_clean = np.nan_to_num(data, nan=0.0, posinf=1e15, neginf=-1e15)
    # Recortar valores extremos
    data_clean = np.clip(data_clean, -1e15, 1e15)
    return data_clean

# Limpiar y ajustar el scaler solo en los datos de entrenamiento
X_train_reshaped_clean = clean_data_for_scaling(X_train_reshaped)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped_clean)

# Volver a la forma original
X_train_scaled = X_train_scaled.reshape(n_samples_train, n_timesteps, n_features)

# Aplicar la misma transformación a los datos de validación y prueba
n_samples_val, _, _ = X_val.shape
X_val_reshaped = X_val.reshape(n_samples_val * n_timesteps, n_features)
# Limpiar datos de validación antes de transformar
X_val_reshaped_clean = clean_data_for_scaling(X_val_reshaped)
X_val_scaled = scaler.transform(X_val_reshaped_clean)
X_val_scaled = X_val_scaled.reshape(n_samples_val, n_timesteps, n_features)

n_samples_test, _, _ = X_test.shape
X_test_reshaped = X_test.reshape(n_samples_test * n_timesteps, n_features)
# Limpiar datos de prueba antes de transformar
X_test_reshaped_clean = clean_data_for_scaling(X_test_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped_clean)
X_test_scaled = X_test_scaled.reshape(n_samples_test, n_timesteps, n_features)

# Normalizar también la variable objetivo
y_scaler = StandardScaler()
# Limpiar datos de la variable objetivo antes de escalar
y_train_clean = clean_data_for_scaling(y_train.reshape(-1, 1))
y_train_scaled = y_scaler.fit_transform(y_train_clean).flatten()

# Aplicar la misma transformación a los datos de validación y prueba
y_val_clean = clean_data_for_scaling(y_val.reshape(-1, 1))
y_val_scaled = y_scaler.transform(y_val_clean).flatten()

y_test_clean = clean_data_for_scaling(y_test.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test_clean).flatten()


/Users/facundobautistabarbera/PycharmProjects/Analisis-Exploratorio/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [ ]:
# --- 10. ARQUITECTURA DEL MODELO LSTM --------------------------------------
def create_lstm_model(input_shape, units=64, dropout_rate=0.2):
    """
    Crea un modelo LSTM para predicción de series temporales.

    Args:
        input_shape: Forma de los datos de entrada (time steps, features)
        units: Número de unidades en la capa LSTM
        dropout_rate: Tasa de dropout para regularización

    Returns:
        Modelo compilado
    """
    model = Sequential([
        # Primera capa LSTM con retorno de secuencias para apilar capas LSTM
        LSTM(units, return_sequences=True, input_shape=input_shape),
        BatchNormalization(),
        Dropout(dropout_rate),

        # Segunda capa LSTM
        LSTM(units // 2, return_sequences=False),
        BatchNormalization(),
        Dropout(dropout_rate),

        # Capas densas para la salida
        Dense(units // 4, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate / 2),

        # Capa de salida (una sola predicción)
        Dense(1)
    ])

    # Compilar modelo
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )

    return model

# Crear modelo
input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
model = create_lstm_model(input_shape)
model.summary()


In [ ]:
# --- 11. ENTRENAMIENTO DEL MODELO --------------------------------------
# Callbacks para mejorar el entrenamiento
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.0001,
        verbose=1
    ),
    ModelCheckpoint(
        '../models/lstm_weekly_best.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

# Entrenar modelo
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# Guardar el modelo final
model.save('../models/lstm_weekly_final.h5')
print("Modelo guardado en '../models/lstm_weekly_final.h5'")


In [ ]:
# --- 12. VISUALIZACIÓN DEL ENTRENAMIENTO --------------------------------------
# Graficar pérdida durante el entrenamiento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('MAE durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# --- 13. EVALUACIÓN DEL MODELO --------------------------------------
# Predecir en conjunto de prueba
y_pred_scaled = model.predict(X_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled).flatten()

# Calcular métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("\n=== EVALUACIÓN DEL MODELO LSTM ===")
print(f"Test MAE: {mae:.2f}")
print(f"Test RMSE: {rmse:.2f}")
print(f"Test R²: {r2:.4f}")
print(f"Test MAPE: {mape:.4f}")

# Comparar con un modelo baseline (predicción con la media)
y_pred_baseline = np.full_like(y_test, y_test.mean())
r2_baseline = 0  # Por definición, R² de predecir la media es 0
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)

print(f"\nMejora sobre baseline:")
print(f"MAE Reduction: {(mae_baseline - mae) / mae_baseline:.2%}")
print(f"R² Improvement: {r2 - r2_baseline:.4f}")


In [ ]:
# --- 14. VISUALIZACIÓN DE RESULTADOS --------------------------------------
# Visualizar predicciones vs valores reales
plt.figure(figsize=(12, 10))

# Predicciones vs valores reales
plt.subplot(2, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales')

# Histograma de errores
plt.subplot(2, 2, 2)
errors = y_pred - y_test
plt.hist(errors, bins=50)
plt.xlabel('Error de Predicción')
plt.ylabel('Frecuencia')
plt.title('Distribución de Errores')

# Errores vs valores reales
plt.subplot(2, 2, 3)
plt.scatter(y_test, errors, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='-')
plt.xlabel('Valores Reales')
plt.ylabel('Error')
plt.title('Errores vs Valores Reales')

# Predicciones a lo largo del tiempo para un cliente aleatorio
plt.subplot(2, 2, 4)
# Seleccionar un cliente aleatorio del conjunto de prueba
random_client = np.random.choice(test_clients)
client_indices = [i for i, client_id in enumerate(test_client_map) if client_id == random_client]

if client_indices:
    client_y_test = y_test[client_indices]
    client_y_pred = y_pred[client_indices]
    plt.plot(range(len(client_y_test)), client_y_test, 'b-', label='Real')
    plt.plot(range(len(client_y_pred)), client_y_pred, 'r-', label='Predicción')
    plt.xlabel('Semana')
    plt.ylabel('Gasto')
    plt.title(f'Predicciones para Cliente ID: {random_client}')
    plt.legend()

plt.tight_layout()
plt.show()

# Análisis por segmentos
print("\nAnálisis por segmentos:")
# Crear segmentos basados en el gasto real
y_test_df = pd.DataFrame({'spend_next': y_test, 'prediction': y_pred})
y_test_df['spend_segment'] = pd.qcut(y_test_df['spend_next'], 4, labels=['Bajo', 'Medio-Bajo', 'Medio-Alto', 'Alto'])

# Calcular métricas por segmento
segment_metrics = y_test_df.groupby('spend_segment').apply(
    lambda x: pd.Series({
        'MAE': mean_absolute_error(x['spend_next'], x['prediction']),
        'RMSE': np.sqrt(mean_squared_error(x['spend_next'], x['prediction'])),
        'R²': r2_score(x['spend_next'], x['prediction']) if len(x) > 1 else np.nan,
        'Count': len(x)
    })
)
print(segment_metrics)


In [ ]:
# --- 15. FUNCIONES DE PREDICCIÓN -------------------------------------------
def prepare_client_sequence(client_id, panel_data, feature_columns, seq_length=6):
    """
    Prepara la secuencia de datos más reciente para un cliente específico.

    Args:
        client_id: ID del cliente
        panel_data: DataFrame con todos los datos
        feature_columns: Lista de columnas de características
        seq_length: Longitud de la secuencia

    Returns:
        Secuencia de datos preparada para el modelo LSTM
    """
    # Filtrar datos del cliente y ordenar por semana
    client_data = panel_data[panel_data['id'] == client_id].sort_values('week')

    if len(client_data) < seq_length:
        return None

    # Tomar las últimas 'seq_length' semanas
    recent_data = client_data.tail(seq_length)

    # Extraer características
    features = recent_data[feature_columns].values

    # Reshape para el modelo LSTM
    sequence = features.reshape(1, seq_length, len(feature_columns))

    return sequence

def predict_client_next_week(model, client_id, panel_data, feature_columns, scaler, y_scaler, seq_length=6):
    """
    Predice el gasto de la próxima semana para un cliente específico.

    Args:
        model: Modelo LSTM entrenado
        client_id: ID del cliente
        panel_data: DataFrame con todos los datos
        feature_columns: Lista de columnas de características
        scaler: Scaler para normalizar características
        y_scaler: Scaler para desnormalizar la predicción
        seq_length: Longitud de la secuencia

    Returns:
        Predicción del gasto para la próxima semana
    """
    # Preparar secuencia
    sequence = prepare_client_sequence(client_id, panel_data, feature_columns, seq_length)

    if sequence is None:
        return None

    # Normalizar secuencia
    sequence_reshaped = sequence.reshape(seq_length, len(feature_columns))
    sequence_scaled = scaler.transform(sequence_reshaped)
    sequence_scaled = sequence_scaled.reshape(1, seq_length, len(feature_columns))

    # Predecir
    prediction_scaled = model.predict(sequence_scaled)

    # Desnormalizar predicción
    prediction = y_scaler.inverse_transform(prediction_scaled)[0][0]

    return float(prediction)

# Calcular medianas por segmento para clientes con pocos datos
segment_medians = panel.groupby("id_estado")["spend_next"].median().to_dict()

def safe_predict(client_id):
    """
    Función de predicción segura que maneja clientes con historial limitado.

    Args:
        client_id: ID del cliente

    Returns:
        Predicción del gasto para la próxima semana
    """
    hist_len = panel[panel["id"] == client_id]["spend"].count()

    # Para datos semanales, requerimos al menos 6 semanas de historial
    if hist_len < 6:
        # Si no hay suficiente historial, usar la mediana del segmento
        segment = panel.loc[panel["id"] == client_id, "id_estado"].iat[0]
        return segment_medians.get(segment, 0)

    # Predecir con el modelo LSTM
    prediction = predict_client_next_week(
        model, client_id, panel_encoded, feature_cols, 
        scaler, y_scaler, seq_length=6
    )

    # Si la predicción falla, usar la mediana del segmento
    if prediction is None:
        segment = panel.loc[panel["id"] == client_id, "id_estado"].iat[0]
        return segment_medians.get(segment, 0)

    return prediction


In [ ]:
# --- 16. SERVIDOR FASTAPI ----------------------------------------------
app = FastAPI()

@app.get("/forecast/{client_id}")
def forecast(client_id: int):
    """Endpoint para predecir el gasto de la próxima semana de un cliente"""
    y_hat = safe_predict(client_id)
    return {"client_id": client_id, "next_week_spend": y_hat}


In [ ]:
# --- 17. GUARDAR RESULTADOS DE EVALUACIÓN --------------------------------------
# Guardar resultados de evaluación
evaluation_results = {
    'MAE': float(mae),
    'RMSE': float(rmse),
    'R²': float(r2),
    'MAPE': float(mape),
    'Segment_Metrics': segment_metrics.to_dict()
}

import json
with open('../models/lstm_weekly_evaluation.json', 'w') as f:
    json.dump({k: v for k, v in evaluation_results.items() if not isinstance(v, pd.DataFrame)}, f, indent=4)
print("\nResultados de evaluación guardados en '../models/lstm_weekly_evaluation.json'")


In [ ]:
# --- 18. COMPARACIÓN CON MODELO LGBM --------------------------------------
try:
    # Intentar cargar resultados de evaluación del modelo LightGBM
    with open('../models/weekly_model_evaluation.json', 'r') as f:
        lgbm_results = json.load(f)

    # Crear tabla comparativa
    comparison = pd.DataFrame({
        'Métrica': ['MAE', 'RMSE', 'R²', 'MAPE'],
        'LSTM': [mae, rmse, r2, mape],
        'LightGBM': [
            lgbm_results.get('MAE', 'N/A'),
            lgbm_results.get('RMSE', 'N/A'),
            lgbm_results.get('R²', 'N/A'),
            lgbm_results.get('MAPE', 'N/A')
        ]
    })

    print("\n=== COMPARACIÓN DE MODELOS ===")
    print(comparison)

    # Visualizar comparación
    plt.figure(figsize=(10, 6))
    metrics = ['MAE', 'RMSE', 'MAPE']
    x = np.arange(len(metrics))
    width = 0.35

    lstm_values = [mae, rmse, mape]
    lgbm_values = [
        lgbm_results.get('MAE', 0),
        lgbm_results.get('RMSE', 0),
        lgbm_results.get('MAPE', 0)
    ]

    plt.bar(x - width/2, lstm_values, width, label='LSTM')
    plt.bar(x + width/2, lgbm_values, width, label='LightGBM')

    plt.xlabel('Métrica')
    plt.ylabel('Valor')
    plt.title('Comparación de Modelos')
    plt.xticks(x, metrics)
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Comparar R² en un gráfico separado
    plt.figure(figsize=(8, 5))
    r2_values = [r2, lgbm_results.get('R²', 0)]
    plt.bar(['LSTM', 'LightGBM'], r2_values)
    plt.xlabel('Modelo')
    plt.ylabel('R²')
    plt.title('Comparación de R² entre Modelos')
    plt.ylim(0, max(r2_values) * 1.2)

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"No se pudo cargar los resultados del modelo LightGBM: {e}")


In [ ]:
# --- 19. CONCLUSIONES --------------------------------------
"""
# Conclusiones del Modelo LSTM

## Rendimiento del Modelo
- El modelo LSTM ha sido entrenado para predecir el gasto semanal de los clientes.
- Se utilizaron secuencias de 6 semanas para capturar patrones temporales.
- Las métricas de evaluación muestran un rendimiento [comparar con LightGBM según resultados].

## Ventajas del Enfoque LSTM
- Capacidad para capturar dependencias temporales a largo plazo.
- Manejo efectivo de la estacionalidad y tendencias en los datos.
- Robustez ante valores atípicos gracias a la normalización y arquitectura del modelo.

## Limitaciones
- Requiere suficiente historial (al menos 6 semanas) para hacer predicciones precisas.
- Mayor complejidad computacional comparado con modelos más simples.
- Sensibilidad a la calidad y completitud de los datos históricos.

## Recomendaciones
- Considerar un enfoque de ensamblaje combinando LSTM con LightGBM para mejorar la precisión.
- Explorar arquitecturas más complejas como redes bidireccionales o atención para mejorar el rendimiento.
- Implementar un sistema de monitoreo para detectar degradación del modelo con el tiempo.
"""
